In [1]:
import sqlite3,random
from typing import Dict, Tuple
import networkx as nx
from networkx.algorithms import approximation as approx
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2

VARIABLES D'ENTRADA

In [2]:
velocidad_media_camiones = 100  # km/h
capacidad_camiones = 1000      # unidades
coste_medio_km = 0.6           # € por km

DEFINIR LES FUNCIONS

In [3]:
# FUNCIO PER SABER LA DISTANCIA ENTRE COORDENADES
def haversine(lat1:float, lon1:float, lat2:float, lon2:float):
    # Radio de la Tierra en km
    R = 6371.0
    
    # Convertir grados a radianes
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    # Diferencias
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distancia en kilómetros
    distance = R * c
    return distance

def mostrar_ruta_con_costes_acumulados(ruta, grafo):
    coste_total = 0
    ruta_con_costes = []  # Lista para almacenar los nodos con sus costes acumulados

    # Recorrer la ruta para calcular los costes acumulados
    for i in range(len(ruta) - 1):
        nodo1 = ruta[i]
        nodo2 = ruta[i + 1]
        
        # Obtener el peso (distancia) entre los nodos
        peso = grafo[nodo1][nodo2]['weight']
        coste_total += peso
        
        # Formato: (nodo1) --> (nodo2) con su coste acumulado
        ruta_con_costes.append(f"{nodo2} ({coste_total:.2f} €)")

    distancia_total = coste_total * coste_medio_km
    temps_total = distancia_total / velocidad_media_camiones

    # Construir la cadena de la ruta con el nodo inicial y la distancia total
    ruta_str = "\nRUTA ÓPTIMA:\n" + f"{ruta[0]} (inicial) --> " + " --> ".join(ruta_con_costes)
    ruta_str += f"\n\nCoste total: {coste_total:.2f} €"
    ruta_str += f"\nDistancia total: {distancia_total:.2f} km"
    ruta_str += f"\nTemps total: {temps_total:.2f} hores"

    return ruta_str


DEFINIR LES CLASES

In [4]:
class Camion():
    def __init__(self,id_camion:int, id_pedidos:list, destinos:list, id_producto:int, cantidad:int):
        self.id_camion = id_camion
        self.id_pedidos = id_pedidos
        self.destinos = destinos
        self.id_producto = id_producto
        self.cantidad = cantidad

    def __str__(self):
            return (f"Camion ID: {self.id_camion}\n"
                    f"Pedidos: {', '.join(map(str, self.id_pedidos))}\n"
                    f"Destinos: {' | '.join(self.destinos)}\n"
                    f"Producto ID: {self.id_producto}\n"
                    f"Cantidad: {self.cantidad}")

class Pedido():
    def __init__(self, id_pedido:int, coordenadas:str, destino:str, id_producto:int, nombre_producto:str, cantidad:int):
        self.id_pedido = id_pedido
        self.coordenadas = coordenadas
        self.destino = destino
        self.id_producto = id_producto
        self.nombre_producto = nombre_producto
        self.cantidad = cantidad

    def __str__(self):
        return (
            f"Pedido #{self.id_pedido}:\n"
            f"  Producto: {self.nombre_producto} (ID: {self.id_producto})\n"
            f"  Cantidad: {self.cantidad}\n"
            f"  Coordenadas de recogida: {self.coordenadas}\n"
            f"  Destino: {self.destino}\n"
        )

EXTREURE LA INFORMACIÓ DE LA BASE DE DADES

In [5]:
conn = sqlite3.connect(r"C:\Users\joant\OneDrive\Stucom\MasterIA\IA\Projecte 1 - Logistica\logistics.db")
cursor = conn.cursor()

###################################### ESTABLIR LES VARIABLES ########################################
velocidad_media_camiones = 100  # km/h
capacidad_camiones = 1000      # unidades
coste_medio_km = 0.6           # € por km
######################################################################################################

# OBTENIR LES COMANDES PER PRODUCTE
pedidos = cursor.execute("""
    SELECT
        ped.id_pedido,
        ped.id_producto, 
        prod.nombre_producto, 
        ped.destino, 
        COUNT(ped.id_pedido) AS total_pedidos, 
        SUM(ped.cantidad) AS total_cantidad
    FROM pedidos ped
    JOIN productos prod ON ped.id_producto = prod.id_producto
    GROUP BY ped.id_producto, ped.destino
    ORDER BY ped.id_producto, ped.destino
""").fetchall()

ITERAR PER ELS RESULTATS DE LA QUERY PER AGRUPAR TOTES LES COORDENADES D'UN MATEIX PRODUCTE I OBTENIR EL TOTAL DE PRODUCTES

In [6]:
pedidos_ordenados = {} # ID_PRODUCTO : [NOMBRE_PRODUCTO, {DESTINO:COORDENADAS}, [CANTIDAD_TOTAL_PRODUCTO]]


for ped in pedidos:

    pedido = Pedido(
        id_pedido = int(ped[0]),
        coordenadas = str(ped[3].replace(" ","")),
        destino = "",
        id_producto = int(ped[1]),
        nombre_producto = str(ped[2]),
        cantidad = int(ped[5])
    )


    # OBTENIR EL NOM DEL DESTI PER FER ELS NODES
    provincia_desti = cursor.execute("""SELECT provincia FROM destinos WHERE latitud = ? AND longitud = ?""",(pedido.coordenadas.split(",")[0],pedido.coordenadas.split(",")[1])).fetchone()

    pedido.destino = provincia_desti[0]

    if pedido.id_producto in pedidos_ordenados.keys():
        pedidos_ordenados[pedido.id_producto].append(pedido)
    
    elif pedido.id_producto not in pedidos_ordenados.keys():
        pedidos_ordenados[pedido.id_producto] = []
        pedidos_ordenados[pedido.id_producto].append(pedido)

DEFINIR QUINS PRODUCTES VAN A QUIN CAMIÓ

In [ ]:
pedidos_pendientes = {} # ID_PEDIDO : [CANTIDAD,[COORDENADAS]]
camiones = {} # ID_PEDIDO : {CAMION}


for id_producto, pedidos_producto in list(pedidos_ordenados.items()): # id_producto = int , ped = Pedido(id_pedido:int, coordenadas:str, destino:str, id_producto:int, nombre_producto:str, cantidad:int)

    cantidades = {} # ID_PEDIDO : [CANTIDAD,[COORDENADAS]]

    # PER CADA PEDIDO DE CADA ID_PRODUCTE
    for ped_prod in pedidos_producto:

        # SI LA QUANTITAT DEL PEDIDO ES MAJOR A LA CAPACITAT D'UN CAMIÓ
        if ped_prod.cantidad >= capacidad_camiones:

            # MENTRE LA QUANTITAT DE PRODUCTE D'UN PEDIDO ES MAJOR QUE LA CAPACITAT D'UN CAMIÓ
            while ped_prod.cantidad > capacidad_camiones: # 1. prod.cantidad = 2400 capacidad_camiones = 1000 / 2. prod.cantidad = 1400 capacidad_camiones = 1000

                cantidad_restante = ped_prod.cantidad - capacidad_camiones # 1. cantidad_restante = 2400 - 1000 = 1400 / 2. cantidad_restante = 1400 - 1000 = 400

                camion = Camion(
                    id_camion = int(ped_prod.id_pedido / ped_prod.cantidad), # PER ESTABLIR UN VALOR UNIC COM A id_camion
                    id_pedidos = [ped_prod.id_pedido],
                    destinos = [ped_prod.coordenadas],
                    id_producto = id_producto,
                    cantidad = capacidad_camiones
                )

                # AFEGIR EL CAMIÓ CREAT
                camiones[camion.id_camion] = camion

                ped_prod.cantidad = cantidad_restante

        # SI HI HA PEDIDOS A cantidades
        elif len(cantidades) > 0:

            # ITERAR PER EL DICCIONARI cantidades PER VEURE SI HI HA ALGUN PEDIDO QUE ES PUGUI COMPLETAR
            for id_pedido_cantidades in list(cantidades.keys()):

                # SI LA SUMA D'UN PEDIDO DE cantidades I LA QUANTITAT DEL PEDIDO ACTUAL ES MAJOR QUE LA CAPACITAT D'UN CAMIO
                if (ped_prod.cantidad + cantidades[id_pedido_cantidades][1]) >= capacidad_camiones:

                    # OBTENIR LA QUANTITAT SOBRANT
                    cantidad_restante = (ped_prod.cantidad + cantidades[id_pedido_cantidades][1]) - capacidad_camiones

                    # CONVERTIR id_pedido_cantidades EN UNA TUPLA SI NO HO ES 
                    if not isinstance(id_pedido_cantidades, (list, tuple)):
                        id_pedido_cantidades_tuple = (id_pedido_cantidades,)
                        id_pedidos_camion = [ped_prod.id_pedido, *(id_pedido_cantidades_tuple)]
                    else:
                        id_pedidos_camion = [ped_prod.id_pedido, *(id_pedido_cantidades)]

                    # CREAR UN CAMIÓ AMB LA INFORMACIÓ DELS DOS PEDIDOS COMBINATS
                    camion = Camion(
                        id_camion = int(ped_prod.id_pedido / (ped_prod.cantidad + random.randint(00000,99999))), # ID_PRODUCTE ACTUAL ENTRE LA SUMA DE LA QUANTIAT + LA QUANTIAT DEL PEDIDO AMB EL QUE S'ESTA AVALUANT
                        id_pedidos = id_pedidos_camion,
                        destinos = [ped_prod.coordenadas] + cantidades[id_pedido_cantidades][2],
                        id_producto = id_producto,
                        cantidad = capacidad_camiones
                    )

                    # AFEGIR EL CAMIÓ CREAT
                    camiones[camion.id_camion] = camion

                    # AFEGIR EL PEDIDO QUE S'ESTA AVALUANT ACTUALMENT AL DICCIONARI cantidades (SIMULA QUE ELS PRODUCTES RESTANTS SON DEL PEDIDO ACTUAL I EL PEIDDO MES VELL ES COMPLETA
                    ped_prod.cantidad = cantidad_restante
                    cantidades[ped_prod.id_pedido] = [id_producto,ped_prod.cantidad, [ped_prod.coordenadas]]

                    # ELIMINAR EL PEDIDO QUE ESTA A cantidades 
                    cantidades.pop(id_pedido_cantidades)

                    

                # SI LA SUMA NO ES SUPERIOR A LA CAPACITAT DEL CAMIÓ ES COMBINEN ELS DOS PEDIDOS A cantidades
                else:
                    
                    # SUMA DE LES QUANTITATS DEL PEDIDO ACTUAL I ELS PEDIDOS GUARDATS
                    cantidad_sumada = ped_prod.cantidad + cantidades[id_pedido_cantidades][1]

                    # AFEGIR A cantidades LA NOVA COORDENADA DE DESTÍ
                    cantidades[id_pedido_cantidades][2].append(ped_prod.coordenadas)

                    # SI ES UNA TUPLA, DESGLOSAR-LA PER QUE NO QUEDI XXXXX,(YYYYY,ZZZZZ) SINO XXXXX,YYYYY,ZZZZZ
                    if isinstance(id_pedido_cantidades,tuple):

                        # AFEGIR A cantidades EL PEDIDO JUNTAT
                        cantidades[ped_prod.id_pedido,*id_pedido_cantidades] = [id_producto,cantidad_sumada,cantidades[id_pedido_cantidades][2]]
                    else:
                        cantidades[ped_prod.id_pedido,id_pedido_cantidades] = [id_producto,cantidad_sumada,cantidades[id_pedido_cantidades][2]]
                    
                    # ELIMINAR DE CANTIDADES LA COMANDA AMB LA QUE S'HA JUNTAT EL PEDIDO ACTUAL
                    cantidades.pop(id_pedido_cantidades)

                    # ELIMINAR DE cantidades LAS KEYS QUE COMPONEN EL PEDIDO JUNTAT (SI ALGUN PEDIDO ES QUEDA SENSE CAMIÓ AL FINAL DE LA ITERACCIÓ EVITA QUE ES DUPLIQUI)
                    if ped_prod.id_pedido in cantidades.keys():
                        cantidades.pop(ped_prod.id_pedido)
        
        # SI LA QUANTITAT DEL PEDIDO NO ES MAJOR A LA CAPACITAT D'UN CAMIÓ I cantidades ESTA BUIT, AFEGIR A cantidades
        else:
            cantidades[ped_prod.id_pedido] = [id_producto, ped_prod.cantidad, [ped_prod.coordenadas]]

    # AFEGIR A pedidos_pendientes ELS PEDIDOS QUE NO TENEN CAMIÓ
    for id_pedido_cantidad,pedido_cantidad in cantidades.items():

        try:
            pedidos_pendientes[*id_pedido_cantidad] = pedido_cantidad
        except:
            pedidos_pendientes[id_pedido_cantidad] = pedido_cantidad

IMPRIMIR ELS CAMIONS CREATS

In [8]:
for camio in camiones.values():
    print(camio, "\n")

print(f"TOTAL CAMIONES: {len(camiones.keys())}\n")

Camion ID: 713
Pedidos: 5007723, 4595118, 7545871
Destinos: 42.402276,-8.583134 | 40.96337,-5.734527 | 41.830116,0.216181
Producto ID: 1
Cantidad: 1000 

Camion ID: 259
Pedidos: 9383569, 1747475, 6893417, 1972162
Destinos: 41.275076,1.513732 | 39.514124,-0.476974 | 39.752122,-8.865028 | 41.116767,-8.574959
Producto ID: 10
Cantidad: 1000 

Camion ID: 121
Pedidos: 6717756, 8402464, 9383569
Destinos: 41.604913,0.679305 | 41.275076,1.513732 | 41.541969,-4.663801
Producto ID: 10
Cantidad: 1000 

Camion ID: 248
Pedidos: 2116187, 6301673, 6154262
Destinos: 38.690981,-6.795618 | 36.783819,-2.742398 | 37.081185,-7.891208
Producto ID: 11
Cantidad: 1000 

Camion ID: 63
Pedidos: 5843322, 4743007, 2116187
Destinos: 39.514124,-0.476974 | 38.690981,-6.795618 | 39.168104,-3.465384
Producto ID: 11
Cantidad: 1000 

Camion ID: 957
Pedidos: 2862217, 3016017, 7480577, 6576371
Destinos: 42.468846,1.493294 | 37.202971,-3.660833 | 38.544685,-7.915859 | 41.327635,2.115424
Producto ID: 13
Cantidad: 1000 

Camio

COL·LOCAR ELS PEDIDOS PENDIENTES EN CAMIONS

iterar producte per producte i comparar amb productes

In [13]:
# ITERAR PER CADA PEDIDO PENDIENTE
for id_pedido_pendiente1, pedido_pendiente1 in list(pedidos_pendientes.items()):

    # EN CAS QUE LA ID_PEDIDO SIGUI UN INT CONVERTIR A TUPLA
    if isinstance(id_pedido_pendiente1,int):
        id_pedido_pendiente1 = (id_pedido_pendiente1,)

    # INICIAR UN RANKING PER VEURE EL PEDIDO PENDIENTE QUE TE MES COINCIDENCIES AMB UN ALTRE
    ranking_coincidencias = [] # [ coincidencias,(id_pedido_pendiente1,id_pedido_pendiente2) ]

    # ITERAR PER LA LLISTA DE PEDIDOS PENDIENTES PER COMPARAR AMB TOTS
    for id_pedido_pendiente2, pedido_pendiente2 in list(pedidos_pendientes.items()):  

        if isinstance(id_pedido_pendiente2,int):
            id_pedido_pendiente2 = (id_pedido_pendiente2,)
        
        if id_pedido_pendiente1 == id_pedido_pendiente2:
            continue
        
        coincidencias = set(pedido_pendiente1[2]) & set(pedido_pendiente2[2])

        if len(ranking_coincidencias) > 0:
            
            # SI EL NUMERO DE COINCIDENCIAS EN EL RANKING ES MENOR QUE LES NOVES COINCIDENCIES ACTUALITZAR RANKING
            if ranking_coincidencias[0][0] < len(coincidencias):
                
                ranking_coincidencias.clear()

                ranking_coincidencias.append([len(coincidencias),(id_pedido_pendiente1,id_pedido_pendiente2)])

        else:
            ranking_coincidencias.append([len(coincidencias),(id_pedido_pendiente1,id_pedido_pendiente2)])
    
    if ranking_coincidencias[0][0] == 0:

        camion = Camion(
            id_camion = int(pedido_pendiente1[1] / random.randint(00000,99999)), # PER ESTABLIR UN VALOR UNIC COM A id_camion
            id_pedidos = [*(id_pedido_pendiente1)],
            destinos = pedido_pendiente1[2],
            id_producto = pedido_pendiente1[0],
            cantidad = pedido_pendiente1[0]
        )

        camiones[camion.id_camion] = camion

        pedidos_pendientes.pop(*(id_pedido_pendiente1))
    
    else:        

        if (pedido_pendiente1[1] + pedidos_pendientes[ranking_coincidencias[0][1][1]][1]) >= capacidad_camiones:

            suma_cantidad_pendientes = pedido_pendiente1[1] + pedido_pendiente2[1]

            camion = Camion(
                id_camion = int(pedido_pendiente1[1] / pedido_pendiente2[1]), # PER ESTABLIR UN VALOR UNIC COM A id_camion
                id_pedidos = [*(id_pedido_pendiente1), *(id_pedido_pendiente2)],
                destinos = pedido_pendiente1[2] + pedido_pendiente2[2],
                id_producto = pedido_pendiente1[0] + pedido_pendiente2[0], # CONVERTIT A LIST PER PODER GUARDAR MÉS D'UN ID_PEDIDO
                cantidad = pedido_pendiente1[1] + pedido_pendiente2[1]
            )

            camiones[camion.id_camion] = camion

            pedido_pendiente2[1] = suma_cantidad_pendientes - capacidad_camiones

            pedidos_pendientes.pop(id_pedido_pendiente1)

KeyError: (2187268,)